# VacationPy
* In this code, google maps is used to show the humidity at the random cities that were collected in WeatherPy. The weater data that was collected is used to locate several cities that meet an ideal vacation climate. The ideal weather chosen is between 70-83F with <10% cloudiness, wind <10 mph, and humidity < 45%. For each ideal climate location, the nearest hotel is found and marked on a map.

In [287]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#Using some tips from StackOverflow to suppress warnings
#https://stackoverflow.com/questions/14463277/how-to-disable-python-warnings
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [288]:
cities_list = pd.read_csv("../WeatherPY/out_put/cities.csv")
cities_list.head()

,City,City ID,Country,Latitude,Longitude,Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph)
0,Verāval,1253237,IN,20.90,70.37,84.04,81,100,7.78
1,Kaitangata,2208248,NZ,-46.28,169.85,51.01,84,100,4.00
2,Bambous Virieux,1106677,MU,-20.34,57.76,70.36,83,40,8.05
3,Mataura,6201424,NZ,-46.19,168.86,48.33,90,100,3.38
4,Malinovoye Ozero,1499779,RU,51.68,79.78,63.61,80,1,2.75


### Humidity Heatmap

In [289]:
# Access maps with unique API key
gmaps.configure(api_key=g_key) 

#Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
humidity_map = gmaps.figure(layout=figure_layout)

#Creating a list of city coordinates from the cities list
locations = cities_list[["Latitude", "Longitude"]].astype(float)
#Geting the humidity 
humidity = cities_list["Humidity(%)"].astype(float)

#Using city locations and humidity to creat a heat map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
humidity_map.add_layer(heat_layer)

# Displaying heatmap
humidity_map 

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

In [290]:
#My ideal weather is between 70-83F with <10% cloudiness, wind <10 mph, and humidity < 45%.
#Find all the cites in the list with my ideal weather. 
ideal_weather = cities_list.loc[(cities_list["Cloudiness(%)"] < 15) & ((cities_list["Temp(F)"] > 70) & (cities_list["Temp(F)"] < 83)) & (cities_list["Wind Speed(mph)"] < 10) & (cities_list["Humidity(%)"] < 45)]
ideal_weather

,City,City ID,Country,Latitude,Longitude,Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph)
65,Farah,1142263,AF,32.50,62.50,74.77,29,0,7.36
96,Aksu,1529660,CN,41.12,80.26,77.27,22,2,6.22
108,Shubarshi,608270,KZ,48.59,57.19,75.18,28,3,1.45
146,Kasongo-Lunda,2315026,CD,-6.47,16.82,73.56,27,1,0.78
156,Tall Kayf,303750,TR,37.08,41.07,82.09,17,0,3.67
236,Kumul,1529484,CN,42.80,93.45,74.86,20,1,9.60
247,Lüderitz,3355672,NaN,-26.65,15.16,75.20,19,0,8.05
266,New Ulm,5039173,US,44.31,-94.46,79.23,44,1,0.96
294,Balkanabat,161616,TM,39.51,54.37,80.60,26,0,6.51
300,Kogon,1217180,UZ,39.72,64.55,77.00,36,0,6.93


### Hotel Map

In [291]:
#Creating a hotel dataframe and adding a hotel name
hotel_df = ideal_weather
hotel_df["Hotel Name"] =""

#Using google maps API to find a hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Find a hotel near the cities with ideal weather     
for index, row in hotel_df.iterrows():
    
    #Setting the parameters for the hotel search URL
    locations = f"{row['Latitude']}, {row['Longitude']}"
    params = {"location": locations,"types": "lodging","radius": 5000,"key": g_key}
    
    #Assemble the URL and make API request
    hotel_results = requests.get(base_url, params=params)
    hotel = hotel_results.json()
    
    #List the city and the closest hotel
    print(f"Finding a hotel near: {row['City']}.")
    try:
        print(f"Closest lodging is {hotel['results'][0]['name']}.")   
        hotel_df.loc[index, 'Hotel Name'] = hotel['results'][0]['name']
    except:
        print("No lodging close by... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = "N/A"
    print("------------")

Finding a hotel near: Farah.
No lodging close by... skipping.
------------
Finding a hotel near: Aksu.
Closest lodging is Pudong Holiday Hotel.
------------
Finding a hotel near: Shubarshi.
No lodging close by... skipping.
------------
Finding a hotel near: Kasongo-Lunda.
No lodging close by... skipping.
------------
Finding a hotel near: Tall Kayf.
No lodging close by... skipping.
------------
Finding a hotel near: Kumul.
Closest lodging is Laoyutang Resort.
------------
Finding a hotel near: Lüderitz.
Closest lodging is LÜDERITZ NEST HOTEL.
------------
Finding a hotel near: New Ulm.
Closest lodging is Best Western Plus New Ulm.
------------
Finding a hotel near: Balkanabat.
Closest lodging is Nebitchi Hotel.
------------
Finding a hotel near: Kogon.
Closest lodging is Hostel "New Bukhara".
------------
Finding a hotel near: San Ramon.
Closest lodging is San Ramon Marriott.
------------
Finding a hotel near: North Branch.
Closest lodging is AmericInn by Wyndham North Branch.
--------

In [293]:
#Take out the cites that don't have lodging nearby
no_hotel = hotel_df[hotel_df["Hotel Name"] == "N/A"].index
new_hotel_list = hotel_df.drop(no_hotel)
final_hotel_list = new_hotel_list.dropna()
final_hotel_list

,City,City ID,Country,Latitude,Longitude,Temp(F),Humidity(%),Cloudiness(%),Wind Speed(mph),Hotel Name
96,Aksu,1529660,CN,41.12,80.26,77.27,22,2,6.22,Pudong Holiday Hotel
236,Kumul,1529484,CN,42.80,93.45,74.86,20,1,9.60,Laoyutang Resort
266,New Ulm,5039173,US,44.31,-94.46,79.23,44,1,0.96,Best Western Plus New Ulm
294,Balkanabat,161616,TM,39.51,54.37,80.60,26,0,6.51,Nebitchi Hotel
300,Kogon,1217180,UZ,39.72,64.55,77.00,36,0,6.93,"Hostel ""New Bukhara"""
360,San Ramon,5392593,US,37.78,-121.98,79.56,39,1,6.93,San Ramon Marriott
421,North Branch,5039421,US,45.51,-92.98,81.66,42,0,0.36,AmericInn by Wyndham North Branch
493,Acarí,3948613,PE,-15.43,-74.62,74.26,42,0,6.67,Hospedaje Ximens


In [294]:
# Using the template add the hotel info to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in final_hotel_list.iterrows()]

#Get the Hotel locations 
locations =final_hotel_list[["Latitude", "Longitude"]]

In [295]:
# Assign the marker and info box layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the humidity map
humidity_map.add_layer(markers)

#Show the map with the hotels in the ideal weather locations
humidity_map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…